In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
from tqdm import tqdm
from PIL import Image
import time 
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.datasets as datasets
from torchmetrics import Accuracy, Precision, Recall, F1Score

In [3]:
import random
import os
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import Dataset, random_split

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = sorted(os.listdir(data_dir))

        self.images = []
        self.labels = []

        for i, class_name in enumerate(self.classes):
            class_dir = os.path.join(data_dir, class_name)
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                self.images.append(image_path)
                self.labels.append(i)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label




In [4]:
def create_data_loaders(dataset_path, train_size=0.8, batch_size=32, shuffle=True):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(), 
    ])

    dataset = CustomDataset(dataset_path, transform=transform)

    # Define sizes for train, validation, and test sets
    train_size = int(train_size * len(dataset))
    val_size = (len(dataset) - train_size) // 2
    test_size = len(dataset) - train_size - val_size

    # Split dataset into train, validation, and test sets
    train_data, val_data, test_data = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

    # Create DataLoader instances for training, validation, and testing
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


In [5]:
# Define the dataset path
dataset_path = "/kaggle/input/plant-leave-diseases-dataset-with-augmentation/Plant_leave_diseases_dataset_with_augmentation"

# Define batch size for DataLoader
batch_size = 32

# Create DataLoader instances for training, validation, and testing
train_loader, val_loader, test_loader = create_data_loaders(dataset_path, batch_size=batch_size)

# Check the lengths of the loaders
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in val_loader: {len(val_loader)}")
print(f"Number of batches in test_loader: {len(test_loader)}")


Number of batches in train_loader: 1538
Number of batches in val_loader: 193
Number of batches in test_loader: 193


In [6]:

for image, label in test_loader:
    
    min_val = torch.min(image)
    mean_val = torch.mean(image)
    max_val = torch.max(image)
    
    print(min_val.item(), mean_val.item(), max_val.item())
    break


0.0 0.4681517779827118 1.0


In [7]:

class MultiClassCNN(nn.Module):
    def __init__(self):
        super(MultiClassCNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.flatten = nn.Flatten()
        self.fc_block = nn.Sequential(
            nn.Linear(32 * 62 * 62, 512),  # Calculated based on the output size after convolutions
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 39)  # 39 classes
        ) 

    def forward(self, x):
        x = self.conv_block(x)
        x = self.flatten(x)
        x = self.fc_block(x)
        return x 


In [ ]:
# Initialization of the model
model = MultiClassCNN()

# Definition of loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Initialization of metrics
accuracy = Accuracy(task='multiclass', num_classes=39, average='macro')
precision = Precision(task='multiclass', num_classes=39, average='macro')
recall = Recall(task='multiclass', num_classes=39, average='macro')
f1 = F1Score(task='multiclass',num_classes=39, average='macro')
val_accuracy = Accuracy(task='multiclass', num_classes=39, average='macro')  # Initialize validation accuracy outside the loop
epoch_losses= []
val_losses=[]
# Training loop
for epoch in range(10):  # Number of epochs
    start_time = time.time()  # Start time
    
    running_loss = 0.0
    model.train()  # Set model to training mode
    
    for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
            # Reset gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()

            # Update model parameters
            optimizer.step()

            # Update running loss
            running_loss += loss.item()

            # Update metrics
            accuracy.update(outputs, labels)
            precision.update(outputs, labels)
            recall.update(outputs, labels)
            f1.update(outputs, labels)

    
    # Calculate metrics after each epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_losses.append(epoch_loss)  # Store epoch loss
    acc = accuracy.compute()
    prec = precision.compute()
    rec = recall.compute()
    f1_score = f1.compute()
    
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1 Score: {f1_score:.4f}")
    
    # Validation loop
    val_loss = 0.0
    model.eval()  # Set model to evaluation mode
    
    for val_images, val_labels in tqdm(val_loader, desc="Validation"):
        with torch.no_grad():
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            val_accuracy.update(val_outputs, val_labels)
    val_acc = val_accuracy.compute()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)  # Store validation loss
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
    
    # Elapsed time
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Time elapsed for epoch {epoch+1}: {epoch_time:.2f} seconds")

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(epoch_losses) + 1), epoch_losses, label='Train Loss', marker='o')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.grid(True)
plt.show()



Epoch 1: 100%|██████████| 1538/1538 [50:27<00:00,  1.97s/it]


Epoch 1, Loss: 1.0958
Accuracy: 0.6102, Precision: 0.6358, Recall: 0.6102, F1 Score: 0.6197


Validation: 100%|██████████| 193/193 [02:56<00:00,  1.09it/s]


Validation Loss: 0.6574, Validation Accuracy: 0.7468
Time elapsed for epoch 1: 3203.75 seconds


Epoch 2: 100%|██████████| 1538/1538 [44:53<00:00,  1.75s/it]


...

Validation: 100%|██████████| 193/193 [02:02<00:00,  1.58it/s]


Validation Loss: 0.5342, Validation Accuracy: 0.8061
Time elapsed for epoch 5: 2820.44 seconds


Epoch 6: 100%|██████████| 1538/1538 [44:09<00:00,  1.72s/it]


Epoch 6, Loss: 0.1016
Accuracy: 0.8640, Precision: 0.8717, Recall: 0.8640, F1 Score: 0.8676


Validation:   0%|          | 0/193 [00:00<?, ?it/s]